<a href="https://colab.research.google.com/github/lalesafarzade/Recommendation_system_Project/blob/lale/Notebooks/6.Recommendation_system_Content_Based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
sns.set_style('white')
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import pickle
#from configs import api_key

In [2]:
def memory_decreaser(df):
    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)
    #else:
       # df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    
    return df

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Reading files

In [4]:
movie_df=pd.read_csv('/content/drive/MyDrive/movies.csv')
link_df=pd.read_csv('/content/drive/MyDrive/links.csv')
rating_df=pd.read_csv('/content/drive/MyDrive/ratings.csv')
json_metadata=pd.read_json('/content/drive/MyDrive/metadata_updated.json', lines=True)

In [5]:
movie=pd.merge(link_df, movie_df, on="movieId")
#movie.drop(['tmdbId'],axis=1,inplace=True)
df1=json_metadata[['imdbId','directedBy','starring','avgRating']]
mvielense_df = pd.merge(movie, df1, on=["imdbId"],how='inner')
mvielense_df.head(2)

,movieId,imdbId,tmdbId,title,genres,directedBy,starring,avgRating
0,1,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...",3.89146
1,2,113497,8844.0,Jumanji (1995),Adventure|Children|Fantasy,Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...",3.26605


In [6]:
del movie_df
del link_df
del json_metadata
del movie

gc.collect()


88

In [7]:
mvielense_df=memory_decreaser(mvielense_df)

Memory usage after optimization is: 3.81 MB


In [8]:
df = pd.merge(rating_df,mvielense_df,on='movieId')
df.head(1)

,userId,movieId,rating,timestamp,imdbId,tmdbId,title,genres,directedBy,starring,avgRating
0,1,296,5.0,1147880044,110912,680.0,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,Quentin Tarantino,"John Travolta, Samuel L. Jackson, Tim Roth, Am...",4.1876


In [9]:
df=memory_decreaser(df)

Memory usage after optimization is: 1907.31 MB


#### Making a Ratings dataframe with average rating and number of ratings:

In [10]:
ratings = pd.DataFrame(df.groupby('title').mean())[['rating','avgRating']]
ratings['num of ratings'] = pd.DataFrame(df.groupby('title')['rating'].count())
ratings.head(2)

,rating,avgRating,num of ratings
title,,,
"""BLOW THE NIGHT!"" Let's Spend the Night Together (1983)",3.000000,3.00000,1
"""Great Performances"" Cats (1998)",2.896648,2.87205,179


## Filtering Qualified movies by IMDB formula

In [13]:
mean_vote_report= df['avgRating'].mean()
minimum_votes= ratings['num of ratings'].quantile(0.7)
def weighted_rating(x, m=minimum_votes, C=mean_vote_report):
    v = x['num of ratings']
    R = x['avgRating']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

q_movies =ratings.copy().loc[ratings['num of ratings']>=minimum_votes]
q_movies['score'] = q_movies.apply(weighted_rating, axis= 1)
q_movies.sort_values(by= 'score', ascending= False, inplace= True)
q_movies=q_movies.reset_index()
q_movies


,title,rating,avgRating,num of ratings,score
0,Planet Earth II (2016),4.483096,4.46924,1124,4.451297
1,Planet Earth (2006),4.464797,4.45460,1747,4.443158
2,"Shawshank Redemption, The (1994)",4.413576,4.41985,81482,4.419611
3,Band of Brothers (2001),4.398599,4.41643,1356,4.402351
4,"Godfather, The (1972)",4.324336,4.33078,52498,4.330446
...,...,...,...,...,...
17785,Son of the Mask (2005),1.232227,1.24212,633,1.319118
17786,Gigli (2003),1.214380,1.21653,758,1.281910
17787,Glitter (2001),1.125561,1.15201,669,1.227865
17788,From Justin to Kelly (2003),1.027578,1.02160,417,1.147534


## Merging dataframes to have more features

In [14]:
df2=mvielense_df[['title','genres','imdbId','directedBy','starring','avgRating','tmdbId']]
df2[df2['title'].isin(q_movies['title'].to_list())]
movies_df = pd.merge(q_movies,df2[['title','genres','directedBy','starring','tmdbId']],on='title')


In [32]:
movies_df=movies_df[(movies_df["score"]>3)&((movies_df['starring']!="")&(movies_df['directedBy']!="")&(movies_df['genres']!="(nogenreslisted)"))]
res=movies_df[['title','genres','directedBy',"starring","score",'tmdbId']]
res.dropna(inplace=True)
res.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14000 entries, 1 to 14414
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   title       14000 non-null  object 
 1   genres      14000 non-null  object 
 2   directedBy  14000 non-null  object 
 3   starring    14000 non-null  object 
 4   score       14000 non-null  float64
 5   tmdbId      14000 non-null  float64
dtypes: float64(2), object(4)
memory usage: 765.6+ KB


In [33]:
def text_preprocessing(x):
  if isinstance(x, str):
    return x.replace(" ", "").lower()
  else:
    return ''


In [34]:
features = ['directedBy', 'starring', 'genres']

for feature in features:
    res[feature] = res[feature].apply(text_preprocessing)



In [35]:
res['genres'] = res['genres'].str.split('|').str.join(' ')
res['directedBy'] = res['directedBy'].str.split(',').str.join(' ')
res['starring'] = res['starring'].str.split(',').str.join(' ')
def keyword_creater(x):
    
    return x['starring']+ ' ' +x['directedBy'] + ' '+ x['genres']

res['keyword'] = res.apply(keyword_creater, axis=1)
keyword_df=res[['keyword']]
keyword_df.head(2)

,keyword
1,davidattenborough alastairfothergill documentary
2,timrobbins morganfreeman bobgunton williamsadl...


## CountVectorizer

In [36]:
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(keyword_df['keyword'])
count_matrix.shape

(14000, 34018)

## Cosine_similarity

In [37]:

my_cosine= cosine_similarity(count_matrix, count_matrix)
my_cosine.shape

(14000, 14000)

## Recommending

In [41]:
res1=res.sort_values(by='title')
indices = pd.Series(index=res1['title'], data = res1.index).drop_duplicates()
indices

title
$ (Dollars) (1971)                               12620
$5 a Day (2008)                                  12312
$9.99 (2008)                                     11075
'71 (2014)                                        3215
'R Xmas (2001)                                   13773
                                                 ...  
loudQUIETloud: A Film About the Pixies (2006)     2469
¡Three Amigos! (1986)                            12869
À nos amours (1983)                               6236
À nous la liberté (Freedom for Us) (1931)         4461
Я худею (2018)                                    9123
Length: 14000, dtype: int64

In [42]:
def get_recommendations(title, cosine_sim= my_cosine,k=10):
    
    sim_scores = list(enumerate(cosine_sim[indices[title]]))
    sim_scores = sorted(sim_scores, key= lambda x : x[1], reverse= True)
    sim_scores = sim_scores[1:k+1]
    movie_indices = [i[0] for i in sim_scores]
    return res.iloc[movie_indices][['title']]

In [43]:
get_recommendations('Sabrina (1995)',my_cosine,4)

,title
2804,"Great Debaters, The (2007)"
145,Spotlight (2015)
1424,Philadelphia (1993)
3213,The Post (2017)


## Pickling

In [44]:
rse_dict=res.to_dict('records')

In [45]:
indices_dict=indices.to_dict()

In [46]:
my_cosine.shape

(14000, 14000)

In [47]:
pickle.dump(indices_dict,open('indices.pkl',"wb"))
from google.colab import files
files.download('indices.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [48]:
import bz2
import pickle
import _pickle as cPickle

In [49]:
# Pickle a file and then compress it into a file with extension 
def compressed_pickle(title, data):
  with bz2.BZ2File(title + '.pbz2',"wb") as f: 
    cPickle.dump(data, f)

compressed_pickle('my_cosine', my_cosine) 

files.download('my_cosine.pbz2')
#pickle.dump(my_cosine,open('my_cosine.pkl',"wb"), protocol=4)

#files.download('my_cosine.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
# Load any compressed pickle file
#def decompress_pickle(file):
  #data = bz2.BZ2File(file, "rb")
  #data = cPickle.load(data)
  #return data


#data = decompress_pickle('/content/my_cosinemy_cosine.pbz2') 

In [50]:
from google.colab import files
pickle.dump(rse_dict,open('rse_dict.pkl',"wb"), protocol=4)

files.download('rse_dict.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>